<a href="https://colab.research.google.com/github/maxi9113/colab_notebook/blob/main/MLP_Algorith.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np

# 1. Definir la función de activación y su derivada
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# 2. Definir los datos de entrada (X) y salida esperada (y) para XOR
X = np.array([ [0.2, 0.9, 0.4], [0.1, 0.3, 0.5], [0.9, 0.7, 0.8], [0.6, 0.4, 0.3] ])
y = np.array([ [0.7, 0.3], [0.6, 0.4], [0.9, 0.5], [0.2, 0.8] ])
# X = np.array([[0,0],
#               [0,1],
#               [1,0],
#               [1,1]])

# y = np.array([[0],
#               [1],
#               [1],
#               [0]])

# 3. Función para inicializar y entrenar el MLP con arquitectura configurable
def train_mlp_fixed_bias(X, y, hidden_layer_sizes, epochs=100000, learning_rate=0.1, seed=1):
    np.random.seed(seed)

    input_neurons = X.shape[1]
    output_neurons = y.shape[1]

    weights = []
    biases = [] # Esta lista seguirá almacenando los bias, pero serán fijos en 1

    layer_sizes = [input_neurons] + hidden_layer_sizes + [output_neurons]

    print(f"Arquitectura de la red: {layer_sizes[0]} (entrada) -> {' -> '.join(map(str, hidden_layer_sizes))} (ocultas) -> {layer_sizes[-1]} (salida)")

    # Inicializar pesos y sesgos para todas las capas
    for i in range(len(layer_sizes) - 1):
        w = np.random.uniform(low=-1, high=1, size=(layer_sizes[i], layer_sizes[i+1]))
        # MODIFICACIÓN: Los bias ahora se inicializan a 1 y no se actualizarán
        b = np.ones((1, layer_sizes[i+1])) # <-- Aquí se fijan los bias a 1
        weights.append(w)
        biases.append(b) # Aunque no se actualice, lo mantenemos en la estructura para el forward pass



    # 4. Entrenamiento del MLP
    for epoch in range(epochs):
        # Forward Propagation
        layer_outputs = [X]

        for i in range(len(weights)):
            current_input = layer_outputs[-1]
            layer_input_weighted = np.dot(current_input, weights[i]) + biases[i] # Se usa el bias fijo
            layer_output = sigmoid(layer_input_weighted)
            layer_outputs.append(layer_output)

        predicted_output = layer_outputs[-1]

        # Backpropagation
        errors = [None] * len(weights)
        deltas = [None] * len(weights)

        error = y - predicted_output
        errors[-1] = error
        deltas[-1] = error * sigmoid_derivative(predicted_output)

        for i in range(len(weights) - 2, -1, -1):
            error_current_layer = deltas[i+1].dot(weights[i+1].T)
            errors[i] = error_current_layer
            deltas[i] = error_current_layer * sigmoid_derivative(layer_outputs[i+1])

        # Actualizar solo los pesos, los bias permanecen fijos en 1
        for i in range(len(weights)):
            weights[i] += layer_outputs[i].T.dot(deltas[i]) * learning_rate
            # biases[i] += np.sum(deltas[i], axis=0, keepdims=True) * learning_rate # <--- Línea comentada, los bias NO se actualizan

        # Imprimir el error
        if (epoch + 1) % 10000 == 0:
            loss = np.mean(np.abs(error))
            print(f"Epoch {epoch + 1}, Loss: {loss:.4f}")

    # 5. Realizar predicciones después del entrenamiento
    def predict(input_data):
        current_output = input_data
        for i in range(len(weights)):
            current_input_weighted = np.dot(current_output, weights[i]) + biases[i]
            current_output = sigmoid(current_input_weighted)
        return current_output

    print("\n--- Predicciones Finales ---")
    final_predictions = predict(X)
    print(final_predictions)
    print("\nValores esperados:")
    print(y)

    return weights, biases, predict


# --- Ejemplo de uso ---
hidden_layer_config = [2] # Aquí configuras tus capas ocultas y neuronas por capa

# Llama a la nueva función que tiene los bias fijos
trained_weights, trained_biases, predict_function = train_mlp_fixed_bias(
    X, y,
    hidden_layer_sizes=hidden_layer_config,
    epochs=50000,
    learning_rate=0.1
)

Arquitectura de la red: 3 (entrada) -> 2 (ocultas) -> 2 (salida)
Epoch 10000, Loss: 0.0407
Epoch 20000, Loss: 0.0294
Epoch 30000, Loss: 0.0230
Epoch 40000, Loss: 0.0192
Epoch 50000, Loss: 0.0170

--- Predicciones Finales ---
[[0.70926623 0.32086032]
 [0.60359147 0.39595307]
 [0.84295527 0.47620995]
 [0.20387872 0.78663816]]

Valores esperados:
[[0.7 0.3]
 [0.6 0.4]
 [0.9 0.5]
 [0.2 0.8]]
